In [ ]:
cd ..

In [ ]:
import json
from importlib import reload
from km.representations.documents.lda import LDAModel
import km.representations.people.aggregators as agg
import numpy as np

In [ ]:
data = json.load(open("files.json"))
data = list(data.values())

model = LDAModel(5)
aggregator = agg.DocumentAggregator(model)
aggregator.fit(data)

In [ ]:
input_document = ["articifical intelligence computers technology danger"]
aggregator.transform(input_document)

In [ ]:
aggregator._topic_model.explain()